Use PySpark to load all of the data

In [1]:
import pyspark

sc = pyspark.SparkContext()

In [2]:
raw = sc.textFile("../states/*.csv")

splitified = raw.filter(lambda x: x != "state,total_votes,dem_votes,dem_pct,dem_ev,rep_votes,rep_pct,rep_ev,year").map(lambda x: x.replace(",,", ",0,").replace(",*,", ",0,").replace("*", "").replace("--", "0")).map(lambda x: x.split(","))
formatted = splitified.map(lambda x: ((x[0], int(x[8])), (int(x[1]), int(x[2]), float(x[3]), int(x[4]), int(x[5]), float(x[6]), int(x[7]))))

print(splitified.takeSample(False, 5))
print(formatted.takeSample(False, 5))

[['Vermont', '102917', '16124', '15.7', '0', '80498', '78.2', '4', '1924'], ['South Dakota', '327703', '103855', '31.7', '0', '198343', '60.5', '4', '1980'], ['Missouri', '1310095', '574962', '43.9', '0', '648488', '49.5', '18', '1924'], ['Arizona', '480770', '237753', '49.5', '0', '242535', '50.4', '5', '1964'], ['Washington', '1241572', '599298', '48.3', '0', '629273', '50.7', '9', '1960']]
[(('Wisconsin', 1916), (447134, 191363, 42.8, 0, 220822, 49.4, 13)), (('Louisiana', 1956), (617544, 243977, 39.5, 0, 329047, 53.3, 10)), (('Alabama', 1916), (131142, 99409, 75.6, 12, 28809, 21.9, 0)), (('Tennessee', 1948), (550283, 270402, 49.1, 11, 202914, 36.9, 0)), (('Minnesota', 1972), (1741652, 802346, 46.1, 0, 898269, 51.6, 10))]


In [6]:
#ratio of dem, rep, and other
ratios = formatted.mapValues(lambda x: (x[1] / x[0], x[4] / x[0], (x[0]-x[4]-x[1]) / x[0]))

print(ratios.takeSample(False, 5))

alabama = ratios.filter(lambda x: x[0][0] == "Alabama").map(lambda x: (x[0][1], x[1]))

print(alabama.takeSample(False, 5))

[(('Pennsylvania', 1964), (0.6492131984431925, 0.34703806381915486, 0.003748737737652638)), (('Vermont', 1932), (0.41076069499196965, 0.5766097240473061, 0.012629580960724193)), (('Alaska', 1964), (0.659079082353291, 0.340920917646709, 0.0)), (('Wisconsin', 1928), (0.44280613002554015, 0.5351970976494619, 0.02199677232499796)), (('Illinois', 2024), (0.543705743159883, 0.43474955221708017, 0.0215447046230369))]
[(2016, (0.3435794575797364, 0.6208309236440906, 0.03558961877617299)), (1928, (0.5132761134383749, 0.4848763560271667, 0.0018475305344584528)), (1948, (0.0, 0.19038980370267003, 0.80961019629733)), (1936, (0.8638302193338749, 0.12822763142625043, 0.007942149239874666)), (1984, (0.38280781265064545, 0.6054249354760621, 0.011767251873292396))]
